# Atmospheric deposition dataset extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is used to retrieve and concatenate the atmospheric deposition data into a table for publication alongisde the used data.


## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas=2.1.3
* scipy=1.9.0
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* 


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 


## References
* 
## Observations
* 

# Import modules

In [ ]:
import tqdm as tqdm
import os
import warnings
import geopandas as gpd
import os
from collections import defaultdict

# Configurations

In [ ]:
# Only editable variables:
# Relative path to your local directory
PATH = "../.."
# Suppress all warnings
warnings.filterwarnings("ignore")

path_data = r"C:\Users\nascimth\Documents\data\CAMELS_CH_Chem\data"

* #### The users should NOT change anything in the code below here. 

In [ ]:
# Non-editable variables:
PATH_OUTPUT = r"results\Dataset\catchment_aggregated_data\atmospheric_deposition"

# Set the directory:
os.chdir(PATH)

# Import data

In [ ]:
catchments = gpd.read_file("results\Dataset\shapefiles\camels_ch_del\camels_ch_chem_catchment_boundaries.shp")

catchments

In [ ]:
# List of filenames
filenames_rasters = [
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dhno3gas_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh3gas_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dnh4total_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno2gas_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dno3total_2020.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_1990.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2000.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2005.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2010.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2015.tif',
    'Ndep_cog500_241105_Teil1_Teil2/dntotal_2020.tif'
]

# Group files by their prefixes
file_groups = defaultdict(list)

for filename in filenames_rasters:
    prefix = filename.split("/")[-1].split("_")[0]  # Extract prefix (e.g., 'dhno3gas')
    file_groups[prefix].append(filename)

In [ ]:
file_groups.keys()

## Reproject to projected coordinates system


In [ ]:
# Define the target CRS to ETRS89 LAEA
target_crs = 'EPSG:2056'

# Reproject the GeoDataFrame to the target CRS
catchments_unique_reprojected = catchments.to_crs(target_crs)

In [ ]:
# Set the index and adjust to int (instead of float)
catchments_unique_reprojected.set_index("gauge_id", inplace=True)
catchments_unique_reprojected.index = catchments_unique_reprojected.index.astype(int)

## Computation processes


In [ ]:
# Convert DataFrame to GeoDataFrame with valid geometries
catchments_unique_reprojected = gpd.GeoDataFrame(
    catchments_unique_reprojected,
    geometry=catchments_unique_reprojected['geometry'],  # Use the existing geometry column
    crs="EPSG:2056"  # Adjust CRS as necessary
)

In [ ]:
from tqdm import tqdm
import pandas as pd
import rasterio
import numpy as np
from rasterio.mask import geometry_mask
from shapely.geometry import mapping
from rasterio.features import geometry_mask

# Define prefixes for their names based on the order of lecture
prefix_values = ["1990_", "2000_", "2005_", "2010_", "2015_", "2020_"]

compounds = ['dhno3gas', 'dnh3gas', 'dnh4total', 'dno2gas', 'dno3total', 'dntotal']

# Iterate over each unique code
for code in tqdm(catchments_unique_reprojected.index):
    # Initialize an empty DataFrame to store the results for the current code
    code_df = pd.DataFrame()

    for compound in compounds:
        # Initialize a DataFrame for the current compound
        compound_data = []
        compound_df = pd.DataFrame()

        i = 0
        filenames = file_groups[compound]

        for filename in filenames:
            # Create lists to store the results
            avg_values = []

            # Load your raster file
            with rasterio.open(path_data+"\\"+ filename) as src:
                geom = catchments_unique_reprojected.loc[[code]]

                # Check if the geometry is empty or invalid
                geometry = geom['geometry'].iloc[0]

                if geometry is None or geometry.is_empty or not geometry.is_valid:
                    avg_value = np.nan
                else:
                    
                    # Convert geometry to GeoJSON-like dict for rasterio
                    geom_mapping = [mapping(geometry)]
                    
                    # Create a mask for the geometry
                    mask = geometry_mask(geom_mapping, out_shape=src.shape, transform=src.transform, invert=True)

                    # Read the values within the geometry from the raster
                    values = src.read(1, masked=True)
                    values = values[mask]

                # Calculate statistics only if there are valid values in the 'values' array
                if len(values) > 0:
                    avg_value = np.sum(values)
                else:
                    # Handle the case when there are no valid values
                    avg_value = np.nan

                # Append the result for the current year (prefix)
                compound_data.append(avg_value)

            i += 1

        # Add the compound data as a column to the code DataFrame
        code_df[compound] = compound_data

    # Add year prefixes as the index
    code_df.index = [prefix[:-1] for prefix in prefix_values]

    
    code_df.index = code_df.index.astype(int)

    # Generate a full range of years from 1980 to 2019
    full_range = pd.DataFrame(index=range(1990, 2021))

    # Reindex the dataframe to include all years
    code_df_interpolated = code_df.reindex(full_range.index)

    # Interpolate missing values
    code_df_interpolated = code_df_interpolated.interpolate(method='linear')


    code_df_interpolated = code_df_interpolated.round(4)
    code_df_interpolated.index.name = "date"

    code_df_interpolated.to_csv(PATH_OUTPUT + "/camels_ch_chem_atmdepo_"+str(code)+".csv", encoding='latin')

# End